# Uso de AlphaVantage sin tooling para ver el cambio de moneda.

Importamos los módulos como siempre y cargamos las variables de entorno. También utilizamos el mismo modelo de Open AI y la api de Rapid Api.

In [86]:
import requests
import os
import openai
import json
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI and MyFitnessPal API keys
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
RAPIDAPI_KEY = os.environ.get('RAPIDAPI_KEY')

client = OpenAI()
model = "gpt-4o-mini"

Antes de nada vamos a escribir como siempre nuestra función para contactar con la api y conseguir el cambio de moneda:

In [87]:
def get_currency_exchange(from_currency, to_currency):
    url = "https://alpha-vantage.p.rapidapi.com/query"
    headers = {
        "x-rapidapi-host": "alpha-vantage.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY,        
    }
    params = {
        "from_currency": from_currency,
        "to_currency": to_currency,
        "function": "CURRENCY_EXCHANGE_RATE"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API Error: {response.status_code} - {response.text}"}

En este caso vamos a utilizar una técnica llamada Few-Shot Prompting. Esta técnica consiste en proporcionar al modelo ejemplos de lo que queremos que nos devuelva para trabajar con ello. En nuestro caso queremos un json con las monedas que queremos utilizar. Cambiar euro a dolar por ejemplo. Necesitaremos un json tal como este:
```json
{
    "to_currency": "EUR",
    "from_currency": "USD"
}
```
Una vez recibido este json, tendremos que llamar a nuestra función para recuperar el exchange rate y pasárselo al modelo. El modelo deberá responder con esto en ves de un json.

In [163]:
messages = [
    {
        "role": "system",
        "content": """
            You are an assistant that checks the currency exchange of 2 currencies and calculate exchange. The user will provide the 2 currencies in the prompt. 
            First you must reply with a json. If a user does not provide 2 currencies, ask the user to provide them. 
            - Example 1:
                - User: "I want to know the exchange between euros and dollars, please"
                - Response: "{ "from_currency": "EUR", "to_currency": "USD" }"
                - Assistant: "Exchange rate from EUR and USD is: 1.04940000"
                - Response: "The exchange rate from EUR and USD is: 1.04940000"
            - Example 2:
                - User: "I want to convert 1000 yens into pounds."
                - Response: "{ "from_currency": "JPY", "to_currency": "GBP" }"
                - Assistant: "Exchange rate from JPY and GBP is: 0,0052"
                - Response: "1000 yens are 5.2 pounds."
            Then, you will be provided the exchange rate and you must calculate or return the result only, not the json.
        """
    }
]

def chat(user_input):
    messages.append({
        "role": "user",
        "content": user_input
    })
        
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    return chat_completion

Ahora viene lo que es la interacción con el modelo. Preguntaremos al usuario por su email y el modelo devolverá la tool que debemos utilizar.

In [164]:
user_input = input("What do you want to know?").strip()

What do you want to know? Convert 100 euros to dollars.


In [168]:
chat_completion = chat(user_input)

In [169]:
print(chat_completion)

ChatCompletion(id='chatcmpl-AdE0fb0gvzczCfOWOHnrlsMyu97rd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"from_currency": "EUR", "to_currency": "USD"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733912009, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_bba3c8e70b', usage=CompletionUsage(completion_tokens=14, prompt_tokens=279, total_tokens=293, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


El modelo en este caso ha respondido correctamente y ha devuelto un json correcto. Debemos detectarlo y efectuar la llamada como si fuera una llamada a una tool.

In [170]:
# esta función comprueba que el string sea un json.
def is_json(myjson):
  try:
    json.loads(myjson)
  except ValueError as e:
    print(e)
    return False
  return True

if is_json(chat_completion.choices[0].message.content):
    json_data = json.loads(chat_completion.choices[0].message.content)
    exchange_data = get_currency_exchange(json_data['from_currency'], json_data['to_currency'])

    print(f"Checking exchange between {json_data['from_currency']} and {json_data['to_currency']}...")

    print(json.dumps(exchange_data, indent=2))

    exchange_data_msg = [{
        "role": "assistant",
        "content": f"Exchange rate from {json_data['from_currency']} and {json_data['to_currency']} is: {exchange_data['Realtime Currency Exchange Rate']['5. Exchange Rate']}"
    }]
else:
    print("Not a json")
    print(chat_completion.choices[0].message.content)

Checking exchange between EUR and USD...
{
  "Realtime Currency Exchange Rate": {
    "1. From_Currency Code": "EUR",
    "2. From_Currency Name": "Euro",
    "3. To_Currency Code": "USD",
    "4. To_Currency Name": "United States Dollar",
    "5. Exchange Rate": "1.04960000",
    "6. Last Refreshed": "2024-12-11 10:13:31",
    "7. Time Zone": "UTC",
    "8. Bid Price": "1.04957000",
    "9. Ask Price": "1.04962000"
  }
}


In [171]:
completion_payload = {
    "model": model,
    "messages": messages + exchange_data_msg
}
print(completion_payload)

{'model': 'gpt-4o-mini', 'messages': [{'role': 'system', 'content': '\n            You are an assistant that checks the currency exchange of 2 currencies and calculate exchange. The user will provide the 2 currencies in the prompt. \n            First you must reply with a json. If a user does not provide 2 currencies, ask the user to provide them. \n            - Example 1:\n                - User: "I want to know the exchange between euros and dollars, please"\n                - Response: "{ "from_currency": "EUR", "to_currency": "USD" }"\n                - Assistant: "Exchange rate from EUR and USD is: 1.04940000"\n                - Response: "The exchange rate from EUR and USD is: 1.04940000"\n            - Example 2:\n                - User: "I want to convert 1000 yens into pounds."\n                - Response: "{ "from_currency": "JPY", "to_currency": "GBP" }"\n                - Assistant: "Exchange rate from JPY and GBP is: 0,0052"\n                - Response: "1000 yens are 5.

In [172]:
response = client.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)

In [173]:
print(response.choices[0].message.content)

100 euros are 104.96 dollars.
